# 0. Environment setup and dataset preparation

Please follow the instructions in the README file to setup the environment and prepare the dataset.

# 1. Novel view synthesis using PAPR

## 1.1 Training

To synthesize novel views of a scene, we need to train a PAPR model. We provide the configuration files for each scene in the NeRF Synthetic dataset and the Tanks and Temples dataset under the `configs` directory. 

The configuration files specify the training parameters and the dataset paths. Detailed comments are provided in `configs/demo.yml` to help you understand the parameters.

To train a PAPR model, on the lego scene of NeRF Synthetic dataset for example, run the following command:

In [ ]:
!python train.py --opt configs/nerfsyn/lego.yml

You can find the trained model in the `experiments` folder. You can specify the folder name using the `index` parameter in the configuration files. You can also find the training logs and middle plots in the folder during training.

## 1.2 Testing

Once the training is finished, you can synthesize images from the test views by running:

In [ ]:
!python test.py --opt configs/nerfsyn/lego.yml

The generated images and evaluation metrics will be saved under `experiments/{index}/test`.

# 2. Improve the performance for in-the-wild data with exposure control

Whenever people take real life photos with auto-exposure turned on, the exposure of the images can vary significantly across different views, for example the Tanks and Temples dataset. To improve the performance of PAPR on in-the-wild data, we propose a simple yet effective method to control the exposure of the rendered images. We introduce an additional latent code input into our model and finetune it using a technique called [conditional Implicit Maximum Likelihood Estimation (cIMLE)](https://arxiv.org/abs/2004.03590). During test time, we can manipulate the exposure of the rendered image by changing the latent code input.

One can refer to Section 4.4 and Appendix A.8 in the paper for more details.

## 2.1 Finetuning

To finetune the model with exposure control, we need a pretrained model generated by following the instructions in [Section 1.1](##-1.1-Training). We provide a exposure control configuration file for the Caterpillar scene from the Tanks and Temples dataset in `configs/t2/Caterpillar_exposure_control.yml`. You need to specify the path to the pretrained model by setting the `load_path` parameter in the configuration file.

To finetune the model with exposure control, run the following command:



In [ ]:
!python exposure_control_train.py --opt configs/t2/Caterpillar_exposure_control.yml

## 2.2 Testing

Render a view with one exposure level

In [ ]:
!python exposure_control_test.py --opt configs/t2/Caterpillar_exposure_control.yml

## 2.3 Generating images with different exposure levels

You can generate images with different exposure levels by changing the latent code input for a given view. We recommend to increase the `shading_code_scale` to generate images with more diverse exposure levels. You can either change the parameter in the configuration file or specify it in the command line. For example, to generate images with different exposure levels for the Caterpillar scene, run:

In [ ]:
!python exposure_control_random.py --opt configs/t2/Caterpillar_exposure_control.yml --scale 8.0 --num_samples 20 --frame 0

## 2.4 Generating images with continuous exposure changes

After you generate the images with different exposure levels, you can interpolate the latent codes to generate images with intermediate exposure levels. For example, to generate images with intermediate exposure levels for the Caterpillar scene, run:

In [ ]:
!python exposure_control_intrp.py --opt configs/t2/Caterpillar_exposure_control.yml --scale 8 --num_samples 20 --frame 0 --start_index 8 --end_index 11 --num_intrp 10